In [1]:
import json
import os
import requests
import zipfile

from datetime import datetime

In [2]:
os.getcwd()

'd:\\tmp\\_reveng\\audioeffect-qm'

In [15]:
os.chdir('..')

In [3]:
src = 'resae/ugc.json'
d = json.load(open(src))

dst = os.path.splitext(src)[0]
os.makedirs(dst, exist_ok=True)
os.chdir(dst)

for ae in d['ugcAudioEffectItems']:

    ae_id = ae['effectId']
    ae_name = ae['effectName']
    ae_url = ae['effectDownloadLink']
    ae_ts = datetime.strptime(ae['createTime'], '%Y-%m-%d %H:%M:%S').timestamp()
    target = f'{ae_id:04d}-{ae_name}.zip'

    print('Downloading', target)
    response = requests.get(ae_url)
    with open(target, 'wb') as f:
        f.write(response.content)
    os.utime(target, (ae_ts, ae_ts))

    with zipfile.ZipFile(target, 'r') as fzip:
        if len(fzip.namelist()) != 2:
            continue
        print('Extracting', target)
        fzip.extractall()

    try:
        target_new = f'{ae_id:04d}-{ae_name}.qmaep'
        os.rename('AudioEffectPackage/AudioEffect.qmaep', target_new)
        os.utime(target_new, (ae_ts, ae_ts))
        os.remove(target)
    except:
        pass

# os.remove('AudioEffectPackage')
os.chdir('../..')

Extracting 0047-沉浸人声.zip
Extracting 0046-演唱现场.zip
Extracting 0043-超人电话亭.zip
Extracting 0038-绵言细语.zip
Extracting 0039-咚咚鼓声.zip
Extracting 0035-老式收音机.zip
Extracting 0034-超宽环绕音域.zip


In [4]:
src = 'resae/soundbox.json'
d = json.load(open(src))
ts = d['timeStamp']

dst = os.path.splitext(src)[0]
os.makedirs(dst, exist_ok=True)
os.chdir(dst)

for v in d['vendorItems']:

    v_id = v['vendorId']
    v_name = v['vendorName']

    for m in v['modelItems']:

        m_id = m['modelId']
        m_name = m['modelName']
        m_url = m['modelAEPLink']
        target = f'{v_id:03d}-{v_name}_{m_id:04d}-{m_name}.aep'

        print('Downloading', target)
        response = requests.get(m_url)
        with open(target, 'wb') as f:
            f.write(response.content)
        os.utime(target, (ts, ts))

os.chdir('../..')